In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud #pip install -U git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

import numpy as np 
import pandas 
import itertools
import json
import zipfile
import os

In [3]:
corpora_address = "/Users/xiche/Downloads/Compressed/NOW"
now_texts = lucem_illud.loadDavies(corpora_address, num_files=2000)

text-16-11.zip
text-16-12.zip
text-17-01.zip
text-17-02.zip
text-17-03.zip
text-17-04.zip
text-17-05.zip
text-17-06.zip
text-17-07.zip
text-17-08.zip
text-17-09.zip
text-17-10.zip
text-17-11.zip
text-17-12.zip
text-18-01.zip
text-18-02.zip
text-18-03.zip
text-18-04.zip
text-18-05.zip
text-18-06.zip
text-18-07.zip
text-18-08.zip
text-18-09.zip
text-18-10.zip
text-18-11.zip
text-18-12.zip
text-19-01.zip
text-19-02.zip
text-19-03.zip
text-19-04.zip
text-19-05.zip
text-19-06.zip
text-19-07.zip
text-19-08.zip
text-19-09.zip
text_10-01-kus.zip
text_10-02-kvz.zip
text_10-03-ovi.zip
text_10-04-laq.zip
text_10-05-wbb.zip
text_10-06-bwq.zip
text_10-07-uek.zip
text_10-08-lqd.zip
text_10-09-udn.zip
text_10-10-nln.zip
text_10-11-weq.zip
text_10-12-ixv.zip
text_11-01-pct.zip
text_11-02-keb.zip
text_11-03-lup.zip
text_11-04-fpt.zip
text_11-05-gbc.zip
text_11-06-hld.zip
text_11-07-mpk.zip
text_11-08-lag.zip
text_11-09-ptj.zip
text_11-10-qsy.zip
text_11-11-rst.zip
text_11-12-dxg.zip
text_12-01-jgs.zip


# loading data with month and year

In [66]:
def get_source(file_name):
    zfile = zipfile.ZipFile(corpora_address + file_name)
    source = []
    for file in zfile.namelist():
        with zfile.open(file) as f:
            for line in f:
                source.append(line)
    return source

In [67]:
def get_dict(source):
    now_txt_dict = {}
    now_dict = {}
    for news in source:
        news_str = news.decode("utf-8")
        if ("inequality" in news_str):
            try:
                textID_title = news_str.split("<p>")[0].strip("@@")
                l_texts = news_str.split("<p>")[1:]
                if "<h>" in textID_title:
                    textID = textID_title.split("<h>")[0]
                    title = textID_title.split("<h>")[-1]
                else:
                    textID = textID_title
                now_txt_dict[textID.strip()] = " ".join(l_texts)
            except UnicodeDecodeError:
                continue
            if textID.strip() not in now_dict:
                now_dict[textID.strip()] = []
            if textID.strip() in now_dict:
                try:
                    now_dict[textID.strip()].append(now_texts[textID.strip()])
                except KeyError:
                    continue
    return now_txt_dict, now_dict

In [69]:
l_df = []
years = ["text_10", "text_11", "text_12", "text_13", "text_14", "text_15", "text_16", "text-16", "text-17", "text-18", "text-19"]
for zip_name in os.listdir(corpora_address):  
    if zip_name != ".DS_Store":
        #print(zip_name)
        for year in years:
            if year in zip_name:
                source = get_source("/" + zip_name)
                nowtxtDict = get_dict(source)[0]
                nowDF = pandas.DataFrame(nowtxtDict.items(), \
                                      columns = ['id','text'])
                nowDF["month"] = zip_name[8:10]
                nowDF["year"] = zip_name[5:7]
                l_df.append(nowDF)

for df in l_df:
    nowDF = nowDF.append(df)

In [70]:
nowDF = nowDF.drop_duplicates(["text"])
nowDF

,id,text,month,year
0,14652434,Most of the debate has focused on the housing...,10,16
1,14660987,The combined revenues of the largest 10 corpo...,10,16
2,14678637,Crystal Legacy receives funding from the Aust...,10,16
3,14698663,World leaders need to ramp up investment to a...,10,16
4,14707705,"Greenwhich Village , NY , mid 1960s . This is...",10,16
...,...,...,...,...
991,14606901,"Beneficiaries include bankers , a former poli...",09,16
992,14607757,Let 's be clear right from the start : everyo...,09,16
993,14609499,File : The calls for large-scale change acros...,09,16
994,14612653,Despite its many achievements in more than tw...,09,16


In [71]:
for df in l_df:
    nowDF = nowDF.append(df, ignore_index=True)
nowDF = nowDF.drop_duplicates(["text"])
nowDF

,id,text,month,year
0,14652434,Most of the debate has focused on the housing...,10,16
1,14660987,The combined revenues of the largest 10 corpo...,10,16
2,14678637,Crystal Legacy receives funding from the Aust...,10,16
3,14698663,World leaders need to ramp up investment to a...,10,16
4,14707705,"Greenwhich Village , NY , mid 1960s . This is...",10,16
...,...,...,...,...
64892,14606901,"Beneficiaries include bankers , a former poli...",09,16
64893,14607757,Let 's be clear right from the start : everyo...,09,16
64894,14609499,File : The calls for large-scale change acros...,09,16
64895,14612653,Despite its many achievements in more than tw...,09,16


In [72]:
nowDF.to_csv("data/inequality.txt", index=False)